<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Intelligent Analysis of
Biomedical Images - Dr. Mohammad Hossein Rohban
</font>
<br/>
<br/>
Fall 2022

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 
            	<br/>
			</font>
    <br/>
    <br/>
<font size=4>
			<br/><br/>
Deadline:  Aban 16
                <br/><b>
              Cheating is Strongly Prohibited
                </b><br/><br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

# Personal Data


In [ ]:
# Set your student number
student_number = 98000000
Name = ''
Last_Name = ''

# Rules
- You are not allowed to use provided codes that can be found on the internet.
- Do not hesitate to ask questions on Quera, if you have any.

In this problem, you are asked to implement a Residual Neural Network for the X-ray images. You can use either Pytorch or Tensorflow. However, the library suggested in part D is implemented using Tensorflow, and it will be easier to use Tensorflow.

# A) Preprocess (5 points):

Import the libraries.


First mount to your Colab.

Now download the dataset from [here](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia), and upload them to the same folder as this Google Colab is located. (1 point)

Load the data. (1 point)

Hint: See this [link](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html).

Make the data ready for the training. Use data augmentation and transformation. (3 points)

Hint: see this [link](https://pytorch.org/vision/stable/transforms.html).

# B) Model (25 + 5 points):

Implement the model. (5 points)
You can read about the residual neural networks in this [paper] (https://arxiv.org/abs/1512.03385).


Describe what the advantages of this network are over the other others. (5 points)

Use transfer leaning. (5 points)

Train the model. (5 points)

Evaluate your model. Use valid evaluation metrics for it. Analyze the results. (evaluation metrics: 5 points, analysis: 5 points, logical results: 5 points)

# C) Data balancing (5 points)

Explain what you can do to balance the data (if it is imbalanced) or avoid overfitting. (2 points)

Implement your idea. (3 points)

# D) Interpretation (15 points)


Read about interpretability. Describe what the interpretation of the neural networks is. (5 points)

Read about interpretation methods in this [paper](https://arxiv.org/abs/1710.10547) or any other resources. Decide which interpretation method you will use, and explain it. (5 points)

Implement the method and visualize the output of 10 input images. You may use any available libraries for this matter. You may want to check out [this link](https://github.com/sicara/tf-explain). (5 points)

# Resources

Specify other resources (if any) you used for this question.